# <font style="color:blue">How to add TensorBoard logs</font>

In this unit we will learn how to add logs to Tensorboard.

**We will add the following logs to the TensorBoard**

1. Scalars: We can add loss, accuracy etc as scalars.

2. Images: We can add plots or figures.

3. Graphs: We can add a network graph and its input-output. 

4. Histograms: We can add n-d array to get its histogram. For example, in each epoch we can add convolution weights.

5. PR Curves: We can add prediction probability and labels to get precision vs recall curves.

6. Projector: We can add sampled training data to get its embeddings. 


**We will use two experiments of the regularization notebook**

1. Medium-sized model, training without regularization.

2. Medium-sized model, training with regularization.



In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt  # one of the best graphics library for python
plt.style.use('ggplot')

In [3]:
import os
import time

from typing import Iterable
from dataclasses import dataclass

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms

from torch.optim import lr_scheduler

from torch.utils.tensorboard import SummaryWriter

## <font style="color:Blue">1. TensorBoard Dashboard</font>

In [ ]:
%load_ext tensorboard
# %reload_ext tensorboard

%tensorboard --logdir=logs_fashion_mnist

# <font style="color:blue">2. Training Utils</font>

## <font style="color:green">2.1. Get Fashion MNIST data</font>

In [5]:
# Fashion mnist class name
fashion_mnist_classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
                         'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']


In [6]:
def get_random_inputs_labels(inputs, targets, n=100):
    """
    get random inputs and labels
    """

    assert len(inputs) == len(targets)

    rand_indices = torch.randperm(len(targets))
    
    data = inputs[rand_indices][:n]
    
    labels = targets[rand_indices][:n]
    
    class_labels = [fashion_mnist_classes[lab] for lab in labels]
    
    return data, class_labels

## <font style="color:magenta">i. Projector in TensorBoard</font>

We can add sampled training data to get its embeddings (e.g. PCA, T-SNE, etc.).

```
SummaryWriter.add_embedding(mat, metadata=None, label_img=None, global_step=None, tag='default', metadata_header=None)
```

**Parameters:**

- **mat** (`torch.Tensor` or `numpy.array`) – A matrix in which each row is the feature vector of the data point. For example, if we have n images of `3` channel (colored) with width `W`and height `H`, then the data shape will be `n x 3 x H x W`. So the mat input should be `n x 3 * H * W`.

- **metadata** (`list`) – A list of labels of which each element will be converted to string.

- **label_img** (`torch.Tensor`) – Images correspond to each data point.

- **global_step** (`python:int`) – Global step value to record.

- **tag** (`string`) – Name for the embedding.

Get details [here](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_embedding)


Function **`add_data_embedings`** in below cell samples add n-datapoints from the dataset and data to TensorBoard.


**PROJECTOR** tab will look similar to the following after changing colored as labels:


<img src="https://www.learnopencv.com/wp-content/uploads/2020/02/w3-w4-projrctor-tensorboard.png" width=900>

In [7]:
def add_data_embedings(dataset, tb_writer, n=100):
    """
    Add a few inputs and labels to tensorboard. 
    """
    
    images, labels = get_random_inputs_labels(inputs=dataset.data, targets=dataset.targets, n=n)
    
    tb_writer.add_embedding(mat = images.view(-1, 28 * 28), 
                            metadata=labels, 
                            label_img=images.unsqueeze(1))
    
    return

In [8]:
def get_data(batch_size, data_root, tb_writer, num_workers=1, data_augmentation=False):
    
    # common transforms
    common_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.2860, ), (0.3530, ))
    ])
    
    # if data_augmentation is true 
    # data augmentation implementation
    if data_augmentation:
        train_transforms = transforms.Compose([
            transforms.RandomChoice([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(90, fill=(0,)),
                transforms.RandomCrop(28, padding=4, fill=(0,))
            ]),
            transforms.ToTensor(),
            transforms.Normalize((0.2860, ), (0.3530, ))
        ])
    # else do common transforms
    else:
        train_transforms = common_transforms
        
        
    
    # train dataloader
    traindata = datasets.FashionMNIST(root=data_root, train=True, download=True, transform=train_transforms)
    
    train_loader = torch.utils.data.DataLoader(
        traindata,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    )
    
    # test dataloader
    testdata = datasets.FashionMNIST(root=data_root, train=False, download=True, transform=common_transforms)
    
    test_loader = torch.utils.data.DataLoader(
        testdata,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers
    )
    
    # add embedding / projector
    
    add_data_embedings(testdata, tb_writer, n=100)
    return train_loader, test_loader

## <font style="color:green">2.2. System Configuration</font>

In [9]:
@dataclass
class SystemConfiguration:
    '''
    Describes the common system setting needed for reproducible training
    '''
    seed: int = 21  # seed number to set the state of all random number generators
    cudnn_benchmark_enabled: bool = True  # enable CuDNN benchmark for the sake of performance
    cudnn_deterministic: bool = True  # make cudnn deterministic (reproducible training)

## <font style="color:green">2.3. Training Configuration</font>

In [10]:
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 32  
    epochs_count: int = 50  
    init_learning_rate: float = 0.02  # initial learning rate for lr scheduler
    decay_rate: float = 0.1  
    log_interval: int = 500  
    test_interval: int = 1  
    data_root: str = "../resource/lib/publicdata/images" 
    num_workers: int = 10  
    device: str = 'cuda'  
    


## <font style="color:green">2.4. System Setup</font>

In [11]:
def setup_system(system_config: SystemConfiguration) -> None:
    torch.manual_seed(system_config.seed)
    if torch.cuda.is_available():
        torch.backends.cudnn_benchmark_enabled = system_config.cudnn_benchmark_enabled
        torch.backends.cudnn.deterministic = system_config.cudnn_deterministic

## <font style="color:green">2.5. Predictions</font>

In [12]:
def prediction(model, device, batch_input, max_prob=True):
    """
    get prediction for batch inputs
    """
    
    # send model to cpu/cuda according to your system configuration
    model.to(device)
    
    # it is important to do model.eval() before prediction
    model.eval()

    data = batch_input.to(device)

    output = model(data)

    # get probability score using softmax
    prob = F.softmax(output, dim=1)
    
    if max_prob:
        # get the max probability
        pred_prob = prob.data.max(dim=1)[0]
    else:
        pred_prob = prob.data
    
    # get the index of the max probability
    pred_index = prob.data.max(dim=1)[1]
    
    return pred_index.cpu().numpy(), pred_prob.cpu().numpy()

In [14]:
def get_target_and_prob(model, dataloader, device):
    """
    get targets and prediction probabilities
    """
    
    pred_prob = []
    targets = []
    
    for _, (data, target) in enumerate(dataloader):
        
        _, prob = prediction(model, device, data, max_prob=False)
        
        pred_prob.append(prob)
        
        target = target.numpy()
        targets.append(target)
        
    targets = np.concatenate(targets)
    targets = targets.astype(int)
    pred_prob = np.concatenate(pred_prob, axis=0)
    
    return targets, pred_prob
    
    

## <font style="color:magenta">ii. PR Curves in TensorBoard</font>

Precision-recall curve tells us about the model’s performance under different threshold settings. With this function we have to provide the ground truth labeling (T/F) and prediction confidence (usually the output of the model) for each target. The TensorBoard UI will let you choose the threshold interactively.
```
SummaryWriter.add_pr_curve(tag, labels, predictions, global_step=None, num_thresholds=127, weights=None, walltime=None)
```


**Parameters:**

- **tag** (`string`) – Data identifier.

- **labels** (`torch.Tensor`, `numpy.array`, or `string/blobname`) – Ground truth data. Binary label for each element.

- **predictions** (`torch.Tensor`, `numpy.array`, or `string/blobname`) – The probability that an element be classified as true. Value should in [0, 1].

- **global_step** (`python:int`) – Global step value to record.

- **num_thresholds** (`python:int`) – Number of thresholds used to draw the curve.

- **walltime** (`python:float`) – Optional override default walltime (time.time()) in seconds.

Get more details [here](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_pr_curve).

Function **`add_pr_curves_to_tensorboard`**, gets target and prediction probabilities from the function `get_target_and_prob` and pass it to `SummaryWriter.add_pr_curve` to get precision-recall curve in TensorBoard.

**PR CURVES** tab will look similar to the following:

<img src="https://www.learnopencv.com/wp-content/uploads/2020/02/c3-w4-prcurves-tensorboard.png" width=900>


In [15]:
def add_pr_curves_to_tensorboard(model, dataloader, device, tb_writer, epoch, num_classes=10):
    """
    Add precession and recall curve to tensorboard.
    """
    
    targets, pred_prob = get_target_and_prob(model, dataloader, device)
    
    for cls_idx in range(num_classes):
        binary_target = targets == cls_idx
        true_prediction_prob = pred_prob[:, cls_idx]
        
        tb_writer.add_pr_curve(fashion_mnist_classes[cls_idx], 
                               binary_target, 
                               true_prediction_prob, 
                               global_step=epoch)
        
    return
    

## <font style="color:magenta">iii. Images in TensorBoard</font>

**Add image data to the summary.**

```
SummaryWriter.add_image(tag, img_tensor, global_step=None, walltime=None, dataformats='CHW')
```

**Parameters:**

- **tag** (`string`) – Data identifier.

- **img_tensor** (`torch.Tensor`, `numpy.array`, or `string/blobname`) – Image data. e.g Matplotlib figures, images, etc.

- **global_step** (`python:int`) – Global step value to record.

- **walltime** (`python:float`) – Optional override default walltime (`time.time()`) in seconds.

Find details [here](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_image).

In the function **`add_wrong_prediction_to_tensorboard`**, we will find wrong predictions, plot as a figure and then add this figure to TensorBoard.

The following is a sample image, which is added to TensorBoard.

<img src="https://www.learnopencv.com/wp-content/uploads/2020/02/c3-w4-images-tensorboard.png" width=900>

In [16]:
def add_wrong_prediction_to_tensorboard(model, dataloader, device, tb_writer, 
                                        epoch, tag='Wrong_Predections', max_images='all'):
    """
    Add wrong predicted images to tensorboard.
    """
    #number of images in one row
    num_images_per_row = 8
    im_scale = 3
    
    plot_images = []
    wrong_labels = []
    pred_prob = []
    right_label = []
    
    for _, (data, target) in enumerate(dataloader):
        
        
        images = data.numpy()
        pred, prob = prediction(model, device, data)
        target = target.numpy()
        indices = pred.astype(int) != target.astype(int)
        
        plot_images.append(images[indices])
        wrong_labels.append(pred[indices])
        pred_prob.append(prob[indices])
        right_label.append(target[indices])
        
    plot_images = np.concatenate(plot_images, axis=0).squeeze()
    wrong_labels = np.concatenate(wrong_labels)
    wrong_labels = wrong_labels.astype(int)
    right_label = np.concatenate(right_label)
    right_label = right_label.astype(int)
    pred_prob = np.concatenate(pred_prob)
    
    
    if max_images == 'all':
        num_images = len(images)
    else:
        num_images = min(len(plot_images), max_images)
        
    fig_width = num_images_per_row * im_scale
    
    if num_images % num_images_per_row == 0:
        num_row = num_images/num_images_per_row
    else:
        num_row = int(num_images/num_images_per_row) + 1
        
    fig_height = num_row * im_scale
        
    plt.style.use('default')
    plt.rcParams["figure.figsize"] = (fig_width, fig_height)
    fig = plt.figure()
    
    for i in range(num_images):
        plt.subplot(num_row, num_images_per_row, i+1, xticks=[], yticks=[])
        plt.imshow(plot_images[i], cmap='gray')
        plt.gca().set_title('{0}({1:.2}), {2}'.format(fashion_mnist_classes[wrong_labels[i]], 
                                                          pred_prob[i], 
                                                          fashion_mnist_classes[right_label[i]]))
        
    tb_writer.add_figure(tag, fig, global_step=epoch)
    
    return


## <font style="color:green">2.6. Training Function</font>

In this section we will train the model. We are already familiar with the training pipeline used in PyTorch.

In [17]:
def train(
    train_config: TrainingConfiguration, model: nn.Module, optimizer: torch.optim.Optimizer,
    train_loader: torch.utils.data.DataLoader, epoch_idx: int, tb_writer: SummaryWriter
) -> None:
    
    # change model in training mood
    model.train()
    
    # to get batch loss
    batch_loss = np.array([])
    
    # to get batch accuracy
    batch_acc = np.array([])
        
    for batch_idx, (data, target) in enumerate(train_loader):
        
        # clone target
        indx_target = target.clone()
        # send data to device (its is mandatory if GPU has to be used)
        data = data.to(train_config.device)
        # send target to device
        target = target.to(train_config.device)

        # reset parameters gradient to zero
        optimizer.zero_grad()
        
        # forward pass to the model
        output = model(data)
        
        # cross entropy loss
        loss = F.cross_entropy(output, target)
        
        # find gradients w.r.t training parameters
        loss.backward()
        # Update parameters using gardients
        optimizer.step()
        
        batch_loss = np.append(batch_loss, [loss.item()])
        
        # Score to probability using softmax
        prob = F.softmax(output, dim=1)
            
        # get the index of the max probability
        pred = prob.data.max(dim=1)[1]  
                        
        # correct prediction
        correct = pred.cpu().eq(indx_target).sum()
            
        # accuracy
        acc = float(correct) / float(len(data))
        
        batch_acc = np.append(batch_acc, [acc])

        if batch_idx % train_config.log_interval == 0 and batch_idx > 0:
            
            total_batch = epoch_idx * len(train_loader.dataset)/train_config.batch_size + batch_idx
            tb_writer.add_scalar('Loss/train-batch', loss.item(), total_batch)
            tb_writer.add_scalar('Accuracy/train-batch', acc, total_batch)
            
    epoch_loss = batch_loss.mean()
    epoch_acc = batch_acc.mean()
    return epoch_loss, epoch_acc

## <font style="color:green">2.7. Validation Function</font>

In [18]:
def validate(
    train_config: TrainingConfiguration,
    model: nn.Module,
    test_loader: torch.utils.data.DataLoader
) -> float:
    # 
    model.eval()
    test_loss = 0
    count_corect_predictions = 0
    for data, target in test_loader:
        indx_target = target.clone()
        data = data.to(train_config.device)
        
        target = target.to(train_config.device)
        
        output = model(data)
        # add loss for each mini batch
        test_loss += F.cross_entropy(output, target).item()
        
        # get probability score using softmax
        prob = F.softmax(output, dim=1)
        
        # get the index of the max probability
        pred = prob.data.max(dim=1)[1] 
        
        # add correct prediction count
        count_corect_predictions += pred.cpu().eq(indx_target).sum()

    # average over number of mini-batches
    test_loss = test_loss / len(test_loader)  
    
    # average over number of dataset
    accuracy = 100. * count_corect_predictions / len(test_loader.dataset)
    
    return test_loss, accuracy/100.0

## <font style="color:magenta">iv. Histogram in TensorBoard</font>

**Add histogram to summary.**

```
SummaryWriter.add_histogram(tag, values, global_step=None, bins='tensorflow', walltime=None, max_bins=None)
```

**Parameters:**

- **tag** (`string`) – Data identifier.

- **values** (`torch.Tensor`, `numpy.array`, or `string/blobname`) – Values to build histogram.

- **global_step** (`python:int`) – Global step value to record.

- **bins** (`string`) – One of {‘tensorflow’,’auto’, ‘fd’, …}. This determines how the bins are made. You can find other options in: https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram.html.

- **walltime** (`python:float`) – Optional override default walltime (`time.time()`) seconds after epoch of event.

Find details [here](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_histogram).

In the function **`add_model_weights_as_histogram`**, we are iterating through named parameters and plotting as a histogram. 

**The followings are the histogram plots for first layer CNN filters for no-regularization and regularization for all epochs:**

---

<table><tr>
<td><img src="https://www.learnopencv.com/wp-content/uploads/2020/02/c3-w4-no-reg-tensorboard.png" alt="Drawing" style="width: 500px;"></td>
<td> <img src="https://www.learnopencv.com/wp-content/uploads/2020/02/c3-w4-reg-tensorboard.png" alt="Drawing" style="width: 500px;"></td>
</tr></table>

---
    
From the histogram, we can observe that the number of weights for no-regularization is lower than regularization

In [19]:
def add_model_weights_as_histogram(model, tb_writer, epoch):
    """
    Get named parameters and plot as histogram
    """
    for name, param in model.named_parameters():
        tb_writer.add_histogram(name.replace('.', '/'), param.data.cpu().abs(), epoch)
    return

## <font style="color:magenta">v. Graph in TensorBoard</font>

**Add network graph and input shape to the summary.**

```
SummaryWriter.add_graph(model, input_to_model=None, verbose=False)
```

**Parameters:**

- **model** (`nn.modules`) – PyTorch model.

- **input_to_model** (`tensor`) – Input tensor.

In the function **`add_network_graph_tensorboard`**, we add a neural network graph and it's inputs.

**The followings are images of the graph and it's inputs**

---


<table><tr>
<td><img src="https://www.learnopencv.com/wp-content/uploads/2020/02/c3-w4-graph-tensorboard.png" alt="Drawing" style="width: 500px;"></td>
<td> <img src="https://www.learnopencv.com/wp-content/uploads/2020/02/c3-w4-inputs-tensorboard.png" alt="Drawing" style="width: 5
    00px;"></td>
</tr></table>


In [20]:
def add_network_graph_tensorboard(model, inputs, tb_writer):
    tb_writer.add_graph(model, inputs)
    return

## <font style="color:green">2.8. Main Function for Training and Validation</font>

## <font style="color:magenta">vi. Scalar In TensorBoard</font>

**Add scalar data to the summary. e.g. loss, accuracy etc.**

```
SummaryWriter.add_scalar(tag, scalar_value, global_step=None, walltime=None)
```

**Parameters:**

- **tag** (`string`) – Data identifier.

- **scalar_value** (`python:float` or `string/blobname`) – Value to save.

- **global_step** (`python:int`) – Global step value to record.

- **walltime** (`python:float`) – Optional override default walltime (`time.time()`) in seconds.

Get details [here](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_scalar).

In the **`main`** function, we add loss, accuracy etc. as a scalar. 

The following is a sample plot of scalar (validation accuracy): 

<img src="https://www.learnopencv.com/wp-content/uploads/2020/02/c3-w4-scalar-tensorboard.png">

## <font style="color:magenta">vii. Scalars in TensorBoard</font>

**Add many scalar data to the summary. e.g. validation and train loss together.**

<font style="color:red">Note that this function also keeps logged scalars in memory. In extreme case it explodes your RAM.</font>

```
SummaryWriter.add_scalars(main_tag, tag_scalar_dict, global_step=None, walltime=None)
```

**Parameters:**

- **main_tag** (`string`) – The parent name for the tags.

- **tag_scalar_dict** (`dict`) – Key-value pair storing the tag and the corresponding values, e.g `dict`.

- **global_step** (`python:int`) – Global step value to record.

- **walltime** (`python:float`) – Optional override default walltime (`time.time()`) in seconds.

Get details [here](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_scalars).

In the **`main`** function, We will add loss and accuracy of training and validation simaltanously as scalars. 

The following is a sample plot of scalar (training and validation loss): 

<img src="https://www.learnopencv.com/wp-content/uploads/2020/02/c3-w4-scalars-tensorboard.png">

In [21]:
def main(model, optimizer, tb_writer, scheduler=None, system_configuration=SystemConfiguration(), 
         training_configuration=TrainingConfiguration(), data_augmentation=False):
    
    # system configuration
    setup_system(system_configuration)

    # batch size
    batch_size_to_set = training_configuration.batch_size
    # num_workers
    num_workers_to_set = training_configuration.num_workers
    # epochs
    epoch_num_to_set = training_configuration.epochs_count

    # if GPU is available use training config, 
    # else lower batch_size, num_workers and epochs count
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
        batch_size_to_set = 16
        num_workers_to_set = 2

    # data loader
    train_loader, test_loader = get_data(
        batch_size=batch_size_to_set,
        data_root=training_configuration.data_root,
        tb_writer=tb_writer,
        num_workers=num_workers_to_set,
        data_augmentation=data_augmentation
    )
    
    
    # Update training configuration
    training_configuration = TrainingConfiguration(
        device=device,
        batch_size=batch_size_to_set,
        num_workers=num_workers_to_set
    )
        
    # send model to device (GPU/CPU)
    model.to(training_configuration.device)
    
    
    # add network graph with inputs info
    images, labels = next(iter(test_loader))
    images = images.to(training_configuration.device)
    add_network_graph_tensorboard(model, images, tb_writer)

    best_loss = torch.tensor(np.inf)
    
    # epoch train/test loss
    epoch_train_loss = np.array([])
    epoch_test_loss = np.array([])
    
    # epoch train/test accuracy
    epoch_train_acc = np.array([])
    epoch_test_acc = np.array([])
    
    
    # training time measurement
    t_begin = time.time()
    for epoch in range(training_configuration.epochs_count):
        
        # Train
        train_loss, train_acc = train(training_configuration, model, optimizer, train_loader, epoch, tb_writer)
        
        epoch_train_loss = np.append(epoch_train_loss, [train_loss])
        
        epoch_train_acc = np.append(epoch_train_acc, [train_acc])
        
        # add scalar (loss/accuracy) to tensorboard
        tb_writer.add_scalar('Loss/Train',train_loss, epoch)
        tb_writer.add_scalar('Accuracy/Train', train_acc, epoch)

        elapsed_time = time.time() - t_begin
        speed_epoch = elapsed_time / (epoch + 1)
        speed_batch = speed_epoch / len(train_loader)
        eta = speed_epoch * training_configuration.epochs_count - elapsed_time
        
        # add time metadata to tensorboard
        tb_writer.add_scalar('Time/elapsed_time', elapsed_time, epoch)
        tb_writer.add_scalar('Time/speed_epoch', speed_epoch, epoch)
        tb_writer.add_scalar('Time/speed_batch', speed_batch, epoch)
        tb_writer.add_scalar('Time/eta', eta, epoch)

        # Validate
        if epoch % training_configuration.test_interval == 0:
            current_loss, current_accuracy = validate(training_configuration, model, test_loader)
            
            epoch_test_loss = np.append(epoch_test_loss, [current_loss])
        
            epoch_test_acc = np.append(epoch_test_acc, [current_accuracy])
            
            # add scalar (loss/accuracy) to tensorboard
            tb_writer.add_scalar('Loss/Validation', current_loss, epoch)
            tb_writer.add_scalar('Accuracy/Validation', current_accuracy, epoch)
            
            # add scalars (loss/accuracy) to tensorboard
            tb_writer.add_scalars('Loss/train-val', {'train': train_loss, 
                                           'validation': current_loss}, epoch)
            tb_writer.add_scalars('Accuracy/train-val', {'train': train_acc, 
                                               'validation': current_accuracy}, epoch)
            
            if current_loss < best_loss:
                best_loss = current_loss
                
            # add wrong predicted image to tensorboard
            add_wrong_prediction_to_tensorboard(model, test_loader, 
                                                training_configuration.device, 
                                                tb_writer, epoch, max_images=300)
        
        # scheduler step/ update learning rate
        if scheduler is not None:
            scheduler.step()
            
        # adding model weights to tensorboard as histogram
        add_model_weights_as_histogram(model, tb_writer, epoch)
        
        # add pr curves to tensor board
        add_pr_curves_to_tensorboard(model, test_loader, 
                                     training_configuration.device, 
                                     tb_writer, epoch, num_classes=10)
        
                
    print("Total time: {:.2f}, Best Loss: {:.3f}".format(time.time() - t_begin, best_loss))
    
    
    
    return model, epoch_train_loss, epoch_train_acc, epoch_test_loss, epoch_test_acc

## <font style="color:green">2.9. Optimizer and Scheduler</font>

In [22]:
def get_optimizer_and_scheduler(model):
    train_config = TrainingConfiguration()

    init_learning_rate = train_config.init_learning_rate

    # optimizer
    optimizer = optim.SGD(
        model.parameters(),
        lr = init_learning_rate,
        momentum = 0.9
    )

    decay_rate = train_config.decay_rate

    lmbda = lambda epoch: 1/(1 + decay_rate * epoch)

    # Scheduler
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lmbda)
    
    return optimizer, scheduler
    


# <font style="color:blue">3. Model</font>

In [23]:
class MediumModel(nn.Module):
    def __init__(self, dropout=0.0, batch_norm=False):
        super().__init__()

        # convolution layers
        if batch_norm:
            self._body = nn.Sequential(
                nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5),
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True),

                nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2),

                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2),
                nn.Dropout(dropout)
            )
        else:
             self._body = nn.Sequential(
                nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5),
                nn.ReLU(inplace=True),

                nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2),

                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2),
                nn.Dropout(dropout)
            )
            
        
        # Fully connected layers
        self._head = nn.Sequential(
            
            nn.Linear(in_features=64 * 4 * 4, out_features=512), 
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            
            nn.Linear(in_features=512, out_features=128), 
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            
            nn.Linear(in_features=128, out_features=10)
        )

    def forward(self, x):
        x = self._body(x)
        x = x.view(x.size()[0], -1)
        x = self._head(x)
        return x

## <font style="color:green">3.1. Experiment 1: No Regularization</font>

In [ ]:
model = MediumModel()

# get optimizer and scheduler
optimizer, scheduler = get_optimizer_and_scheduler(model)

# Tensorboard summary writer
no_regularization_sw = SummaryWriter('logs_fashion_mnist/no_regularization')   

# train and validate
model, train_loss_exp2, train_acc_exp2, val_loss_exp2, val_acc_exp2 = main(model, 
                                                                           optimizer,
                                                                           no_regularization_sw,
                                                                           scheduler)
no_regularization_sw.close()

## <font style="color:green">3.2. Experiment 2: Regularization</font>

In [ ]:
model = MediumModel(0.25, batch_norm=True)

optimizer, scheduler = get_optimizer_and_scheduler(model)

# Tensorboard summary writer
regularization_sw = SummaryWriter('logs_fashion_mnist/regularization')  

model, train_loss_exp9, train_acc_exp9, val_loss_exp9, val_acc_exp9 = main(model, 
                                                                           optimizer, 
                                                                           regularization_sw,
                                                                           scheduler,
                                                                           data_augmentation=True)

regularization_sw.close()

# <font style="color:Blue">4. Exercise</font>

1. Add CNN layers as images to TensorBoard.

2. Add CNN layers output after activation and see what it has learned. 


# <font style="color:Blue">References</font>

- https://pytorch.org/docs/stable/tensorboard.html

- https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html
